# 07 — Gun Homicide vs Gun Control Strictness

Scatter plot, strip plot, and trend line exploring the correlation between gun control
strictness (custom ordinal scale 1-5) and gun homicide rates.

**Scale:** 1 = Very Permissive, 2 = Permissive, 3 = Moderate, 4 = Strict, 5 = Very Strict

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from scipy import stats
from pathlib import Path

DATA_DIR = Path('../data/processed')
df = pd.read_csv(DATA_DIR / 'merged_country_data.csv')

# Filter to countries with both gun control strictness and gun homicide data
plot_df = df.dropna(subset=['gun_control_strictness', 'gun_homicide_rate']).copy()
print(f"Countries with both gun control strictness and gun homicide data: {len(plot_df)}")

STRICTNESS_LABELS = {
    1: '1 - Very Permissive',
    2: '2 - Permissive',
    3: '3 - Moderate',
    4: '4 - Strict',
    5: '5 - Very Strict',
}
plot_df['strictness_label'] = plot_df['gun_control_strictness'].map(STRICTNESS_LABELS)

Countries with both gun control strictness and gun homicide data: 160


## Scatter Plot — Gun Homicide Rate vs Gun Control Strictness

In [2]:
# Linear regression on log-transformed homicide rate
plot_df['log_homicide'] = np.log10(plot_df['gun_homicide_rate'].clip(lower=0.01))

slope, intercept, r_value, p_value, std_err = stats.linregress(
    plot_df['gun_control_strictness'], plot_df['log_homicide']
)
r_squared = r_value ** 2

print(f"Linear regression (gun control strictness vs log10 gun homicide rate):")
print(f"  R² = {r_squared:.4f}")
print(f"  p-value = {p_value:.2e}")
print(f"  slope = {slope:.4f}")

Linear regression (gun control strictness vs log10 gun homicide rate):
  R² = 0.2085
  p-value = 1.29e-09
  slope = -0.4222


In [3]:
# Add jitter for visibility
np.random.seed(42)
plot_df['strictness_jittered'] = plot_df['gun_control_strictness'] + np.random.uniform(-0.15, 0.15, len(plot_df))

fig = px.scatter(
    plot_df,
    x='strictness_jittered',
    y='gun_homicide_rate',
    color='region',
    hover_name='country_name',
    hover_data={'gun_control_strictness': True, 'gun_homicide_rate': ':.2f',
                'region': True, 'strictness_jittered': False},
    log_y=True,
    title=f'Gun Homicide Rate vs Gun Control Strictness (R²={r_squared:.3f}, p={p_value:.2e})',
    labels={
        'strictness_jittered': 'Gun Control Strictness (1=Very Permissive → 5=Very Strict)',
        'gun_homicide_rate': 'Gun Homicide Rate per 100K (log scale)',
        'region': 'Region',
    },
)

# Add trend line
x_range = np.linspace(1, 5, 100)
y_trend = 10 ** (slope * x_range + intercept)
fig.add_trace(go.Scatter(
    x=x_range, y=y_trend,
    mode='lines',
    name=f'Trend (R²={r_squared:.3f})',
    line=dict(color='red', dash='dash', width=2),
))

fig.update_layout(
    template='plotly_white', height=600,
    xaxis=dict(tickmode='array', tickvals=[1, 2, 3, 4, 5],
               ticktext=['1 - Very\nPermissive', '2 - Permissive',
                         '3 - Moderate', '4 - Strict', '5 - Very\nStrict']),
)
fig.show()

## Strip / Box Plot — Homicide Rate Distribution by Strictness Level

In [4]:
fig = px.strip(
    plot_df,
    x='strictness_label',
    y='gun_homicide_rate',
    color='region',
    hover_name='country_name',
    hover_data={'gun_homicide_rate': ':.2f', 'region': True, 'strictness_label': False},
    log_y=True,
    title='Gun Homicide Rate Distribution by Gun Control Strictness Level',
    labels={
        'strictness_label': 'Gun Control Strictness',
        'gun_homicide_rate': 'Gun Homicide Rate per 100K (log scale)',
        'region': 'Region',
    },
    category_orders={'strictness_label': list(STRICTNESS_LABELS.values())},
)

# Overlay box plot
for level in sorted(plot_df['gun_control_strictness'].unique()):
    subset = plot_df[plot_df['gun_control_strictness'] == level]
    fig.add_trace(go.Box(
        y=subset['gun_homicide_rate'],
        x=[STRICTNESS_LABELS[level]] * len(subset),
        name=f'Box {level}',
        marker_color='rgba(0,0,0,0.3)',
        line_color='rgba(0,0,0,0.5)',
        boxpoints=False,
        showlegend=False,
    ))

fig.update_layout(template='plotly_white', height=600)
fig.show()

## Gun Control Strictness Choropleth Map

In [5]:
control_map = df.dropna(subset=['gun_control_strictness'])

fig = px.choropleth(
    control_map,
    locations='country_code',
    color='gun_control_strictness',
    hover_name='country_name',
    hover_data={'gun_control_strictness': True, 'country_code': False},
    color_continuous_scale='RdYlGn',
    range_color=[1, 5],
    title='Gun Control Strictness by Country (1=Very Permissive → 5=Very Strict)',
    labels={'gun_control_strictness': 'Strictness'},
)
fig.update_layout(
    geo=dict(showframe=False, showcoastlines=True, coastlinecolor='#999'),
    template='plotly_white',
    height=500,
    coloraxis_colorbar=dict(
        tickvals=[1, 2, 3, 4, 5],
        ticktext=['1 Very Permissive', '2 Permissive', '3 Moderate', '4 Strict', '5 Very Strict'],
    ),
)
fig.show()